In [2]:
from Model.CustomerModel import CustomerModel
from Model.SalesOrderModel import SalesOrderModel
from Connector.DatabaseConnection import DatabaseConnection

server = "localhost"
port = 3306
username="root"
database="northwind"
password="hoanganh123"

connector =DatabaseConnection(server=server, port=port, password=password, username=username, database=database)
sales_order_df = SalesOrderModel(connector).dataframe

{'employeeId': {'referenced_column': 'employeeId', 'model': <Model.EmployeeModel.EmployeeModel object at 0x00000157B613FEB0>}, 'shipperid': {'referenced_column': 'shipperId', 'model': <Model.ShipperModel.ShipperModel object at 0x00000157B613FF40>}, 'custId': {'referenced_column': 'custId', 'model': <Model.CustomerModel.CustomerModel object at 0x00000157B613FBE0>}}


In [3]:
sales_order_df.head()

,orderId,custId,employeeId,orderDate,requiredDate,shippedDate,shipperid,freight,shipName,shipAddress,shipCity,shipRegion,shipPostalCode,shipCountry
0,10248,85,5,04-Jul-2006,01-Aug-2006,16-Jul-2006,3,32.38,Ship to 85-B,6789 rue de l'Abbaye,Reims,None,10345,France
1,10249,79,6,05-Jul-2006,16-Aug-2006,10-Jul-2006,1,11.61,Ship to 79-C,Luisenstr. 9012,Münster,None,10328,Germany
2,10250,34,4,08-Jul-2006,05-Aug-2006,12-Jul-2006,2,65.83,Destination SCQXA,"Rua do Paço, 7890",Rio de Janeiro,RJ,10195,Brazil
3,10251,84,3,08-Jul-2006,05-Aug-2006,15-Jul-2006,1,41.34,Ship to 84-A,"3456, rue du Commerce",Lyon,None,10342,France
4,10252,76,4,09-Jul-2006,06-Aug-2006,11-Jul-2006,2,51.30,Ship to 76-B,"Boulevard Tirou, 9012",Charleroi,None,10318,Belgium
